# ViT-B-16 Testing

In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
%pip install torchinfo
%restart_python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import sys
import os
sys.path.append(os.path.abspath('..'))

from bacp import BaCPLearner, BaCPTrainer, BaCPTrainingArgumentsLLM
from models import EncoderProjectionNetwork, ClassificationNetwork
from unstructured_pruning import MagnitudePrune, MovementPrune, LocalMagnitudePrune, LocalMovementPrune, WandaPrune, PRUNER_DICT, check_model_sparsity
from LLM_trainer import LLMTrainer, LLMTrainingArguments
from CV_trainer import *
from dataset_utils import get_glue_data
from logger import Logger

import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from tqdm import tqdm
from torchinfo import summary

from datasets.utils.logging import disable_progress_bar
disable_progress_bar()
import os
os.environ["HF_DATASETS_CACHE"] = "/dbfs/hf_datasets"
os.environ["TOKENIZERS_PARALLELISM"] = "false" 

from utils import *
from constants import *

device = get_device()
print(f"{device = }")
BATCH_SIZE_VIT = 512


device = 'cuda'


## Baseline Accuracies

In [0]:
# Model initialization
model_name = "vitb16"
model_task = "cifar10"

training_args = CVTrainingArguments(
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_VIT,
    image_size=224,
    epochs=3,
    learning_type="baseline",
)

trainer = Trainer(training_args=training_args)

if False:
    trainer.train()

acc = trainer.evaluate()
print(f"\nAccuracy = {acc}")

[TRAINER] Initializing model


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:01<00:00, 177MB/s]


[TRAINER] Initializing data loaders for cifar10
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Pruning disabled
[TRAINER] Current model sparsity: 0.0
[TRAINER] Saving model checkpoints to /dbfs/research/vitb16/cifar10/vitb16_baseline.pt
[TRAINER] Loading weights: /dbfs/research/vitb16/cifar10/vitb16_baseline.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.00


Evaluating: 100%|██████████| 19/19 [00:12<00:00,  1.52it/s, Acc=97.28%]


Accuracy = 97.28


## Pruning Accuracies

### Magnitude Pruning

In [0]:
# Model initialization
model_name = "vitb16"
model_task = "cifar10"

# Initializing finetuned weights path
finetuned_weights = "/dbfs/research/vitb16/cifar10/vitb16_baseline.pt"

# Initializing pruning args
pruning_type = "magnitude_pruning"
target_sparsity = TARGET_SPARSITY_LOW
learning_type = "pruning"

training_args = CVTrainingArguments(
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_VIT,
    finetuned_weights=finetuned_weights,
    image_size=224,
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    learning_type=learning_type,

    learning_rate=0.0001,
    epochs=5,
    recovery_epochs=10,
    sparsity_scheduler='cubic'
)

trainer = Trainer(training_args=training_args)

if False:
    trainer.train()

acc = trainer.evaluate()
print(f"\nAccuracy = {acc}")

[TRAINER] Initializing model
[TRAINER] Weights loaded
[TRAINER] Initializing data loaders for cifar10
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Pruning enabled
[TRAINER] Initializing pruner for magnitude_pruning
[TRAINER] Pruning scheduler: cubic
[TRAINER] Pruning target sparsity: 0.95
[TRAINER] Current model sparsity: 0.0
[TRAINER] Saving model checkpoints to /dbfs/research/vitb16/cifar10/vitb16_magnitude_pruning_0.95.pt
[TRAINER] Loading weights: /dbfs/research/vitb16/cifar10/vitb16_magnitude_pruning_0.95.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.95


Evaluating: 100%|██████████| 19/19 [00:06<00:00,  2.95it/s, Acc=83.20%]



Accuracy = 83.2


In [0]:
# Model initialization
model_name = "vitb16"
model_task = "cifar10"

# Initializing finetuned weights path
finetuned_weights = "/dbfs/research/vitb16/cifar10/vitb16_baseline.pt"

# Initializing pruning args
pruning_type = "magnitude_pruning"
target_sparsity = TARGET_SPARSITY_MID
learning_type = "pruning"

training_args = CVTrainingArguments(
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_VIT,
    finetuned_weights=finetuned_weights,
    image_size=224,
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    learning_type=learning_type,

    learning_rate=0.0001,
    epochs=5,
    recovery_epochs=10,
    sparsity_scheduler='cubic'
)

trainer = Trainer(training_args=training_args)

if False:
    trainer.train()

acc = trainer.evaluate()
print(f"\nAccuracy = {acc}")

[TRAINER] Initializing model
[TRAINER] Weights loaded
[TRAINER] Initializing data loaders for cifar10
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Pruning enabled
[TRAINER] Initializing pruner for magnitude_pruning
[TRAINER] Pruning scheduler: cubic
[TRAINER] Pruning target sparsity: 0.97
[TRAINER] Current model sparsity: 0.0
[TRAINER] Saving model checkpoints to /dbfs/research/vitb16/cifar10/vitb16_magnitude_pruning_0.97.pt
[TRAINER] Loading weights: /dbfs/research/vitb16/cifar10/vitb16_magnitude_pruning_0.97.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.97


Evaluating: 100%|██████████| 19/19 [00:06<00:00,  3.14it/s, Acc=75.96%]



Accuracy = 75.96


In [0]:
# Model initialization
model_name = "vitb16"
model_task = "cifar10"

# Initializing finetuned weights path
finetuned_weights = "/dbfs/research/vitb16/cifar10/vitb16_baseline.pt"

# Initializing pruning args
pruning_type = "magnitude_pruning"
target_sparsity = TARGET_SPARSITY_HIGH
learning_type = "pruning"

training_args = CVTrainingArguments(
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_VIT,
    finetuned_weights=finetuned_weights,
    image_size=224,
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    learning_type=learning_type,

    learning_rate=0.0001,
    epochs=5,
    recovery_epochs=10,
    sparsity_scheduler='cubic'
)

trainer = Trainer(training_args=training_args)

if False:
    trainer.train()

acc = trainer.evaluate()
print(f"\nAccuracy = {acc}")

[TRAINER] Initializing model
[TRAINER] Weights loaded
[TRAINER] Initializing data loaders for cifar10
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Pruning enabled
[TRAINER] Initializing pruner for magnitude_pruning
[TRAINER] Pruning scheduler: cubic
[TRAINER] Pruning target sparsity: 0.99
[TRAINER] Current model sparsity: 0.0
[TRAINER] Saving model checkpoints to /dbfs/research/vitb16/cifar10/vitb16_magnitude_pruning_0.99.pt
[TRAINER] Loading weights: /dbfs/research/vitb16/cifar10/vitb16_magnitude_pruning_0.99.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.99


Evaluating: 100%|██████████| 19/19 [00:05<00:00,  3.30it/s, Acc=65.56%]



Accuracy = 65.56


### Movement Pruning

In [0]:
# Model initialization
model_name = "vitb16"
model_task = "cifar10"

# Initializing finetuned weights path
finetuned_weights = "/dbfs/research/vitb16/cifar10/vitb16_baseline.pt"

# Initializing pruning args
pruning_type = "movement_pruning"
target_sparsity = TARGET_SPARSITY_LOW
learning_type = "pruning"

training_args = CVTrainingArguments(
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_VIT,
    finetuned_weights=finetuned_weights,
    image_size=224,
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    learning_type=learning_type,

    learning_rate=0.0001,
    epochs=5,
    recovery_epochs=10,
    sparsity_scheduler='cubic'
)

trainer = Trainer(training_args=training_args)

if False:
    trainer.train()

acc = trainer.evaluate()
print(f"\nAccuracy = {acc}")

[TRAINER] Initializing model
[TRAINER] Weights loaded
[TRAINER] Initializing data loaders for cifar10
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Pruning enabled
[TRAINER] Initializing pruner for movement_pruning
[TRAINER] Pruning scheduler: cubic
[TRAINER] Pruning target sparsity: 0.95
[TRAINER] Current model sparsity: 0.0
[TRAINER] Saving model checkpoints to /dbfs/research/vitb16/cifar10/vitb16_movement_pruning_0.95.pt
[TRAINER] Loading weights: /dbfs/research/vitb16/cifar10/vitb16_movement_pruning_0.95.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.95


Evaluating: 100%|██████████| 19/19 [00:06<00:00,  2.91it/s, Acc=91.87%]



Accuracy = 91.87


In [0]:
# Model initialization
model_name = "vitb16"
model_task = "cifar10"

# Initializing finetuned weights path
finetuned_weights = "/dbfs/research/vitb16/cifar10/vitb16_baseline.pt"

# Initializing pruning args
pruning_type = "movement_pruning"
target_sparsity = TARGET_SPARSITY_MID
learning_type = "pruning"

training_args = CVTrainingArguments(
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_VIT,
    finetuned_weights=finetuned_weights,
    image_size=224,
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    learning_type=learning_type,

    learning_rate=0.0001,
    epochs=5,
    recovery_epochs=10,
    sparsity_scheduler='cubic'
)

trainer = Trainer(training_args=training_args)

if False:
    trainer.train()

acc = trainer.evaluate()
print(f"\nAccuracy = {acc}")

[TRAINER] Initializing model
[TRAINER] Weights loaded
[TRAINER] Initializing data loaders for cifar10
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Pruning enabled
[TRAINER] Initializing pruner for movement_pruning
[TRAINER] Pruning scheduler: cubic
[TRAINER] Pruning target sparsity: 0.97
[TRAINER] Current model sparsity: 0.0
[TRAINER] Saving model checkpoints to /dbfs/research/vitb16/cifar10/vitb16_movement_pruning_0.97.pt
[TRAINER] Loading weights: /dbfs/research/vitb16/cifar10/vitb16_movement_pruning_0.97.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.97


Evaluating: 100%|██████████| 19/19 [00:06<00:00,  2.91it/s, Acc=85.87%]



Accuracy = 85.87


In [0]:
# Model initialization
model_name = "vitb16"
model_task = "cifar10"

# Initializing finetuned weights path
finetuned_weights = "/dbfs/research/vitb16/cifar10/vitb16_baseline.pt"

# Initializing pruning args
pruning_type = "movement_pruning"
target_sparsity = TARGET_SPARSITY_HIGH
learning_type = "pruning"

training_args = CVTrainingArguments(
    model_name=model_name,
    model_task=model_task,
    batch_size=BATCH_SIZE_VIT,
    finetuned_weights=finetuned_weights,
    image_size=224,
    pruning_type=pruning_type,
    target_sparsity=target_sparsity,
    learning_type=learning_type,

    learning_rate=0.0001,
    epochs=5,
    recovery_epochs=10,
    sparsity_scheduler='cubic'
)

trainer = Trainer(training_args=training_args)

if False:
    trainer.train()

acc = trainer.evaluate()
print(f"\nAccuracy = {acc}")

[TRAINER] Initializing model
[TRAINER] Weights loaded
[TRAINER] Initializing data loaders for cifar10
[CV DATALOADERS] Loaded cifar10 with splits: ['train', 'validation', 'test']
[TRAINER] Pruning enabled
[TRAINER] Initializing pruner for movement_pruning
[TRAINER] Pruning scheduler: cubic
[TRAINER] Pruning target sparsity: 0.99
[TRAINER] Current model sparsity: 0.0
[TRAINER] Saving model checkpoints to /dbfs/research/vitb16/cifar10/vitb16_movement_pruning_0.99.pt
[TRAINER] Loading weights: /dbfs/research/vitb16/cifar10/vitb16_movement_pruning_0.99.pt
[TRAINER] Weights loaded successfully
[TRAINER] Model Sparsity: 0.99


Evaluating: 100%|██████████| 19/19 [00:06<00:00,  3.01it/s, Acc=77.69%]



Accuracy = 77.69


## BaCP Accuracies